# Data
This notebook generates the data for the campaign - unfortunately the original dataset is a little disjointed. But we are going to use batch inference to fix that and make a very strong dataset for analysis.

This notebook works with Serverless

In [0]:
%pip install dbdemos

In [0]:
import dbdemos
dbdemos.install(
  'aibi-marketing-campaign', 
  catalog='shm', 
  schema='marketing',
  overwrite=True
  )

Now, navigate to `./aibi-marketing-campaign/_genie_spaces/DBDemos - AI-BI - Marketing Campaign` (or click the Genie space link above) and get the Genie ID from the URL, e.g. https://adb-984752964297111.11.azuredatabricks.net/genie/rooms/01f01ae0314e1bee9eb26ea2d75bede6 has the Genie space ID = '01f01ae0314e1bee9eb26ea2d75bede6'. You will need this later on.

We are also going to grab a sample product dataset, saved locally in this repo and use it to generate better reviews using batch inference (via AI_QUERY). AI Query can make a MUCH better campaign table using this product table. We are going to use structured outputs to generate a useful json for a random sample of the products. This query is a bit of a beast but randomly joins 100 products to our campaign table and uses Llama 70B to generate solid campaign slogans for real products

In [0]:
%sql
SELECT * FROM shm.marketing.campaigns LIMIT 5

In [0]:
import pandas as pd
df = spark.createDataFrame(pd.read_csv('products.csv'))
df.write.format('delta').mode('overwrite').saveAsTable('shm.marketing.products')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS shm.marketing.campaigns_fixed AS
-- Get a nice joined table
WITH RandomProduct AS (
  SELECT
      ROW_NUMBER() OVER (ORDER BY RAND()) + 100 AS row,
      *
  FROM shm.marketing.products_sample
  LIMIT 100
)
SELECT 
  C.campaign_id, C.template, C.cost, 
  C.start_date, C.end_date, C.mailing_list,
  P.title, P.product_id, 
  json_tuple(
    AI_QUERY(
      'databricks-meta-llama-3-3-70b-instruct',
      CONCAT('Using the following product title, description, and feature summary, make a witty campaign name, campaign description, and email marketing subject line. Return them in json format. \n', title, product_description, features_summary),
      responseFormat => '{
        "type": "json_schema",
        "json_schema": {
          "name": "generated_campaign",
          "schema": {
            "type": "object",
            "properties": {
              "campaign_name": {"type": "string"},
              "campaign_description": {"type": "string"},
              "subject_line": {"type": "string"}
            }
          }
        }
      }'
    ), 
    'campaign_name', 'campaign_description', 'subject_line') 
    AS (campaign_name, campaign_description, subject_line)
FROM RandomProduct P
JOIN shm.marketing.campaigns C
ON P.row = C.campaign_id

In [0]:
%sql
SELECT * FROM shm.marketing.campaigns_fixed
LIMIT 1

Now that we have a fixed up table, let's go make some tools!